In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd

current_dir = Path(os.getcwd())
sys.path.append(str(current_dir.parent))  # 1階層上を追加

from metrics import get_result

/tmp/ipykernel_2232017/3848427155.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
res = pd.DataFrame()
for i in [
    "nci", "gdsc1", "gdsc2", "ctrp"]:
    true = pd.read_csv(f"true_{i}.csv", index_col=0)
    pred = pd.read_csv(f"pred_{i}.csv", index_col=0)
    res = pd.concat([res, get_result(true, pred, i)], axis=1)

In [3]:
res.T

,ACC,Precision,Recall,Specificity,F1,F2,Balanced ACC,G-Mean,AUROC,AUPR,LogLoss,Brier,MCC,Kappa,Youden J,Cost Ratio
NCI,0.569 (± 0.006),0.600 (± 0.011),0.418 (± 0.047),0.721 (± 0.042),0.491 (± 0.029),0.444 (± 0.041),0.569 (± 0.006),0.547 (± 0.014),0.607 (± 0.007),0.601 (± 0.010),0.675 (± 0.002),0.241 (± 0.001),0.146 (± 0.011),0.139 (± 0.012),0.214 (± 0.014),1.036 (± 0.080)
GDSC1,0.816 (± 0.002),0.945 (± 0.002),0.672 (± 0.005),0.961 (± 0.002),0.785 (± 0.003),0.713 (± 0.004),0.816 (± 0.002),0.803 (± 0.002),0.936 (± 0.002),0.940 (± 0.001),0.435 (± 0.005),0.136 (± 0.001),0.661 (± 0.003),0.633 (± 0.004),0.872 (± 0.004),2.880 (± 0.038)
GDSC2,0.806 (± 0.004),0.968 (± 0.004),0.633 (± 0.006),0.979 (± 0.003),0.765 (± 0.006),0.680 (± 0.006),0.806 (± 0.004),0.787 (± 0.005),0.936 (± 0.011),0.943 (± 0.008),0.467 (± 0.012),0.145 (± 0.004),0.652 (± 0.008),0.612 (± 0.008),0.872 (± 0.021),2.638 (± 0.054)
CTRP,0.815 (± 0.001),0.872 (± 0.001),0.740 (± 0.002),0.891 (± 0.001),0.800 (± 0.001),0.763 (± 0.002),0.815 (± 0.001),0.812 (± 0.001),0.897 (± 0.002),0.898 (± 0.002),0.429 (± 0.003),0.135 (± 0.001),0.638 (± 0.002),0.631 (± 0.002),0.794 (± 0.003),3.349 (± 0.025)
